### Setting up the Data

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier 

from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import matplotlib.pyplot as plt

from sklearn import datasets
np.random.seed(1)

In [2]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score,f1_score

In [3]:
X_test = pd.read_csv("rfm_test_X.csv")
y_test= pd.read_csv("rfm_test_y.csv")
X_train= pd.read_csv("rfm_train_X.csv")
y_train= pd.read_csv("rfm_train_y.csv")

In [4]:
X_train

,recency,monetary,frequency,recency_score,frequency_score,monetary_score
0,49,89,1721.57,3,4,4
1,59,32,1294.14,3,4,3
2,320,10,153.00,1,1,1
3,24,110,781.28,4,3,4
4,8,96,1841.03,5,4,4
...,...,...,...,...,...,...
3155,289,6,94.00,1,1,1
3156,1,17,1393.79,5,4,2
3157,23,119,1228.32,4,4,4
3158,183,10,213.70,1,1,1



### Modelling the Data

In [5]:
performance = pd.DataFrame({"model": [], "Accuracy": [], "Precision": [], "Recall": [], "F1": []})

### Logistic Regression Model

In [6]:
log_reg_model = LogisticRegression()
_ = log_reg_model.fit(X_train, np.ravel(y_train))

C:\Users\Jeshwant\anaconda3\envs\tf\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [7]:
model_preds = log_reg_model.predict(X_test)
performance = pd.concat([performance, pd.DataFrame({'model':"default logistic", 
                                                    'Accuracy': accuracy_score(y_test, model_preds) , 
                                                    'Precision': precision_score(y_test, model_preds, average='weighted') , 
                                                    'Recall': recall_score(y_test, model_preds, average='weighted'), 
                                                    'F1': f1_score(y_test, model_preds, average='weighted') }, index=[0])])
performance

C:\Users\Jeshwant\anaconda3\envs\tf\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,model,Accuracy,Precision,Recall,F1
0,default logistic,0.616456,0.590185,0.616456,0.57152


### Logistic Regression Model(RandomizedSearchCV )

In [8]:
score_measure = "Precision"
LR=LogisticRegression()
kfolds = 4
param_grid = {'C': [0.1, 0.5, 10,0.001], 
              "solver" : [ 'lbfgs', 'liblinear'],
              "penalty" : ['l1','l2','elastic'],
             "warm_start":[True, False]} 
  
grid = RandomizedSearchCV(LR, param_grid, verbose = 1, n_jobs = -1)
  
# fitting the model for grid search
grid.fit(X_train, np.ravel(y_train))
print(f"The best {score_measure} score is {grid.best_score_}")
print(f"... with parameters: {grid.best_params_}")

bestRecallTree = grid.best_estimator_

Fitting 5 folds for each of 10 candidates, totalling 50 fits


C:\Users\Jeshwant\anaconda3\envs\tf\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
20 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
3 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Jeshwant\anaconda3\envs\tf\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Jeshwant\anaconda3\envs\tf\lib\site-packages\sklearn\linear_model\_logistic.py", line 1160, in fit
    self._validate_params()
  File "C:\Users\Jeshwant\anaconda3\envs\tf\lib\site-packages\sklearn\base.py", line 600, in _validate_params
    validate_parameter_constraints(

The best Precision score is 0.9468354430379746
... with parameters: {'warm_start': True, 'solver': 'liblinear', 'penalty': 'l1', 'C': 10}


In [9]:
model_preds = bestRecallTree.predict(X_test)
performance = pd.concat([performance, pd.DataFrame({'model':"Logistic Regression Randomised", 
                                                    'Accuracy': accuracy_score(y_test, model_preds) , 
                                                    'Precision': precision_score(y_test, model_preds, average='weighted') , 
                                                    'Recall': recall_score(y_test, model_preds, average='weighted'), 
                                                    'F1': f1_score(y_test, model_preds, average='weighted') }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.616456,0.590185,0.616456,0.571520
0,Logistic Regression Randomised,0.939241,0.946318,0.939241,0.933195


### Logistic Regression Model(GridSearchCV)

In [10]:
score_measure = "precision"
kfolds = 3
param_grid = {'C': [1,5,0.5], 
              'solver' : [ 'liblinear'],
              'penalty' : ['l1']} 
  
grid = GridSearchCV(LogisticRegression(), param_grid, refit = True, verbose = 3)
  
# fitting the model for grid search
grid.fit(X_train, y_train)
print(f"The best {score_measure} score is {grid.best_score_}")
print(f"... with parameters: {grid.best_params_}")

bestRecallTree = grid.best_estimator_

Fitting 5 folds for each of 3 candidates, totalling 15 fits


C:\Users\Jeshwant\anaconda3\envs\tf\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV 1/5] END .C=1, penalty=l1, solver=liblinear;, score=0.943 total time=   0.3s


C:\Users\Jeshwant\anaconda3\envs\tf\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV 2/5] END .C=1, penalty=l1, solver=liblinear;, score=0.940 total time=   0.3s


...

[CV 4/5] END C=0.5, penalty=l1, solver=liblinear;, score=0.926 total time=   0.2s


C:\Users\Jeshwant\anaconda3\envs\tf\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV 5/5] END C=0.5, penalty=l1, solver=liblinear;, score=0.921 total time=   0.2s


C:\Users\Jeshwant\anaconda3\envs\tf\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


The best precision score is 0.9449367088607594
... with parameters: {'C': 5, 'penalty': 'l1', 'solver': 'liblinear'}


In [11]:
model_preds = bestRecallTree.predict(X_test)
performance = pd.concat([performance, pd.DataFrame({'model':"Logistic Regression Grid", 
                                                    'Accuracy': accuracy_score(y_test, model_preds) , 
                                                    'Precision': precision_score(y_test, model_preds, average='weighted') , 
                                                    'Recall': recall_score(y_test, model_preds, average='weighted'), 
                                                    'F1': f1_score(y_test, model_preds, average='weighted') }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.616456,0.590185,0.616456,0.571520
0,Logistic Regression Randomised,0.939241,0.946318,0.939241,0.933195
0,Logistic Regression Grid,0.930380,0.939948,0.930380,0.918941


In [12]:
performance.sort_values(by =['Recall'])

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.616456,0.590185,0.616456,0.571520
0,Logistic Regression Grid,0.930380,0.939948,0.930380,0.918941
0,Logistic Regression Randomised,0.939241,0.946318,0.939241,0.933195


### Decision Tree Model

In [13]:
from sklearn.tree import DecisionTreeClassifier

# instantiate the DecisionTreeClassifier model 
clf = DecisionTreeClassifier(criterion='entropy', max_depth=3, random_state=0)
# fit the model
clf.fit(X_train, y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=3, random_state=0)

In [14]:
model_preds = clf.predict(X_test)
performance = pd.concat([performance, pd.DataFrame({'model':"Decision_trees", 
                                                    'Accuracy': accuracy_score(y_test, model_preds) , 
                                                    'Precision': precision_score(y_test, model_preds, average='weighted') , 
                                                    'Recall': recall_score(y_test, model_preds, average='weighted'), 
                                                    'F1': f1_score(y_test, model_preds, average='weighted') }, index=[0])])
performance

C:\Users\Jeshwant\anaconda3\envs\tf\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,model,Accuracy,Precision,Recall,F1
0,default logistic,0.616456,0.590185,0.616456,0.571520
0,Logistic Regression Randomised,0.939241,0.946318,0.939241,0.933195
0,Logistic Regression Grid,0.930380,0.939948,0.930380,0.918941
0,Decision_trees,0.901266,0.838693,0.901266,0.863157


### Decision tree model (Randomsearch)

In [15]:
score_measure = "precision"
kfolds = 4

param_grid = {
    'min_samples_split': np.arange(1,80),  
    'min_samples_leaf': np.arange(1,40),
    'min_impurity_decrease': np.arange(0.0001, 0.01, 0.0005),
    'max_leaf_nodes': np.arange(5, 200), 
    'max_depth': np.arange(1,50), 
    'criterion': ['entropy', 'gini'],
}

dtree = DecisionTreeClassifier()
rand_search = RandomizedSearchCV(estimator = dtree, param_distributions=param_grid, cv=kfolds, n_iter=500,
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = rand_search.fit(X_train, y_train)

print(f"The best {score_measure} score is {rand_search.best_score_}")
print(f"... with parameters: {rand_search.best_params_}")

bestRecallTree = rand_search.best_estimator_

Fitting 4 folds for each of 500 candidates, totalling 2000 fits
The best precision score is nan
... with parameters: {'min_samples_split': 36, 'min_samples_leaf': 20, 'min_impurity_decrease': 0.0026, 'max_leaf_nodes': 85, 'max_depth': 41, 'criterion': 'gini'}


C:\Users\Jeshwant\anaconda3\envs\tf\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
12 fits failed out of a total of 2000.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
12 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Jeshwant\anaconda3\envs\tf\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Jeshwant\anaconda3\envs\tf\lib\site-packages\sklearn\tree\_classes.py", line 889, in fit
    super().fit(
  File "C:\Users\Jeshwant\anaconda3\envs\tf\lib\site-packages\sklearn\tree\_classes.py", line 177, in fit
    self._validate_params()
  File "C:\Users\Jeshwant\ana

In [16]:
model_preds = bestRecallTree.predict(X_test)
performance = pd.concat([performance, pd.DataFrame({'model':"Decision_trees_random_Search", 
                                                    'Accuracy': accuracy_score(y_test, model_preds) , 
                                                    'Precision': precision_score(y_test, model_preds, average='weighted') , 
                                                    'Recall': recall_score(y_test, model_preds, average='weighted'), 
                                                    'F1': f1_score(y_test, model_preds, average='weighted') }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.616456,0.590185,0.616456,0.571520
0,Logistic Regression Randomised,0.939241,0.946318,0.939241,0.933195
0,Logistic Regression Grid,0.930380,0.939948,0.930380,0.918941
0,Decision_trees,0.901266,0.838693,0.901266,0.863157
0,Decision_trees_random_Search,0.998734,0.998759,0.998734,0.998733


### Decision tree model (Gridsearch)

In [17]:
score_measure = "precision"
kfolds = 5

param_grid = {
    'min_samples_split': np.arange(45,55),  
    'min_samples_leaf': np.arange(27,32),
    'min_impurity_decrease': np.arange(0.004, 0.005, 0.0001),
    'max_leaf_nodes': np.arange(70,75), 
    'max_depth': np.arange(32,36), 
    'criterion': ['gini'],
}

dtree = DecisionTreeClassifier()
grid_search = GridSearchCV(estimator = dtree, param_grid=param_grid, cv=kfolds, 
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = grid_search.fit(X_train, y_train)

print(f"The best {score_measure} score is {grid_search.best_score_}")
print(f"... with parameters: {grid_search.best_params_}")

bestRecallTree = grid_search.best_estimator_

Fitting 5 folds for each of 10000 candidates, totalling 50000 fits
The best precision score is nan
... with parameters: {'criterion': 'gini', 'max_depth': 32, 'max_leaf_nodes': 70, 'min_impurity_decrease': 0.004, 'min_samples_leaf': 27, 'min_samples_split': 45}


C:\Users\Jeshwant\anaconda3\envs\tf\lib\site-packages\sklearn\model_selection\_search.py:952: UserWarning: One or more of the test scores are non-finite: [nan nan nan ... nan nan nan]
  warnings.warn(
C:\Users\Jeshwant\anaconda3\envs\tf\lib\site-packages\sklearn\model_selection\_search.py:952: UserWarning: One or more of the train scores are non-finite: [nan nan nan ... nan nan nan]
  warnings.warn(


In [20]:
model_preds = bestRecallTree.predict(X_test)
performance = pd.concat([performance, pd.DataFrame({'model':"Decision_trees_Grid_Search", 
                                                    'Accuracy': accuracy_score(y_test, model_preds) , 
                                                    'Precision': precision_score(y_test, model_preds, average='weighted') , 
                                                    'Recall': recall_score(y_test, model_preds, average='weighted'), 
                                                    'F1': f1_score(y_test, model_preds, average='weighted') }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.616456,0.590185,0.616456,0.571520
0,Logistic Regression Randomised,0.939241,0.946318,0.939241,0.933195
0,Logistic Regression Grid,0.930380,0.939948,0.930380,0.918941
0,Decision_trees,0.901266,0.838693,0.901266,0.863157
0,Decision_trees_random_Search,0.998734,0.998759,0.998734,0.998733
0,Decision_trees_Grid_Search,0.997468,0.997504,0.997468,0.997466


# SVM Classification model(Linear Kernel)

In [21]:
svm_lin_model = SVC(kernel="linear", probability=True)
_ = svm_lin_model.fit(X_train, np.ravel(y_train))

In [22]:
model_preds = svm_lin_model.predict(X_test)
performance = pd.concat([performance, pd.DataFrame({'model':"svm_linear", 
                                                    'Accuracy': accuracy_score(y_test, model_preds) , 
                                                    'Precision': precision_score(y_test, model_preds, average='weighted') , 
                                                    'Recall': recall_score(y_test, model_preds, average='weighted'), 
                                                    'F1': f1_score(y_test, model_preds, average='weighted') }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.616456,0.590185,0.616456,0.571520
0,Logistic Regression Randomised,0.939241,0.946318,0.939241,0.933195
0,Logistic Regression Grid,0.930380,0.939948,0.930380,0.918941
0,Decision_trees,0.901266,0.838693,0.901266,0.863157
0,Decision_trees_random_Search,0.998734,0.998759,0.998734,0.998733
0,Decision_trees_Grid_Search,0.997468,0.997504,0.997468,0.997466
0,svm_linear,0.998734,0.998743,0.998734,0.998733


# SVM Classification model(RBF Kernel)

In [23]:
svm_rbf_model = SVC(kernel="rbf", C=10, gamma='scale', probability=True)
_ = svm_rbf_model.fit(X_train, np.ravel(y_train))

In [24]:
model_preds = svm_rbf_model.predict(X_test)
performance = pd.concat([performance, pd.DataFrame({'model':"svm_rbf", 
                                                    'Accuracy': accuracy_score(y_test, model_preds) , 
                                                    'Precision': precision_score(y_test, model_preds, average='weighted') , 
                                                    'Recall': recall_score(y_test, model_preds, average='weighted'), 
                                                    'F1': f1_score(y_test, model_preds, average='weighted') }, index=[0])])
performance

C:\Users\Jeshwant\anaconda3\envs\tf\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,model,Accuracy,Precision,Recall,F1
0,default logistic,0.616456,0.590185,0.616456,0.571520
0,Logistic Regression Randomised,0.939241,0.946318,0.939241,0.933195
0,Logistic Regression Grid,0.930380,0.939948,0.930380,0.918941
0,Decision_trees,0.901266,0.838693,0.901266,0.863157
0,Decision_trees_random_Search,0.998734,0.998759,0.998734,0.998733
0,Decision_trees_Grid_Search,0.997468,0.997504,0.997468,0.997466
0,svm_linear,0.998734,0.998743,0.998734,0.998733
0,svm_rbf,0.631646,0.583086,0.631646,0.591722


# SVM Classification model(Polynomial Kernel)

In [25]:
svm_poly_model = SVC(kernel="poly", degree=3, coef0=1, C=10, probability=True)
_ = svm_poly_model.fit(X_train, np.ravel(y_train))

In [26]:
model_preds = svm_poly_model.predict(X_test)
performance = pd.concat([performance, pd.DataFrame({'model':"svm_poly", 
                                                    'Accuracy': accuracy_score(y_test, model_preds) , 
                                                    'Precision': precision_score(y_test, model_preds, average='weighted') , 
                                                    'Recall': recall_score(y_test, model_preds, average='weighted'), 
                                                    'F1': f1_score(y_test, model_preds, average='weighted') }, index=[0])])
performance

C:\Users\Jeshwant\anaconda3\envs\tf\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,model,Accuracy,Precision,Recall,F1
0,default logistic,0.616456,0.590185,0.616456,0.571520
0,Logistic Regression Randomised,0.939241,0.946318,0.939241,0.933195
0,Logistic Regression Grid,0.930380,0.939948,0.930380,0.918941
0,Decision_trees,0.901266,0.838693,0.901266,0.863157
0,Decision_trees_random_Search,0.998734,0.998759,0.998734,0.998733
0,Decision_trees_Grid_Search,0.997468,0.997504,0.997468,0.997466
0,svm_linear,0.998734,0.998743,0.998734,0.998733
0,svm_rbf,0.631646,0.583086,0.631646,0.591722
0,svm_poly,0.673418,0.654952,0.673418,0.639693


# Neural Networks

In [27]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score

In [28]:
%%time

ann = MLPClassifier(hidden_layer_sizes=(60,50,40),solver='adam', max_iter=200)
_ = ann.fit(X_train, y_train)

C:\Users\Jeshwant\anaconda3\envs\tf\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1098: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


CPU times: total: 4.47 s
Wall time: 1.16 s


In [29]:
%%time
y_pred = ann.predict(X_test)

CPU times: total: 78.1 ms
Wall time: 2.99 ms


In [30]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.72      0.45      0.55        51
           1       0.68      0.77      0.72        99
           2       0.67      0.50      0.57        16
           3       0.93      0.11      0.19       120
           4       0.87      0.95      0.91       186
           5       0.49      0.77      0.60       141
           6       0.42      0.76      0.54        42
           7       1.00      0.45      0.62        11
           8       0.72      0.80      0.76        99
           9       0.88      0.28      0.42        25

    accuracy                           0.67       790
   macro avg       0.74      0.58      0.59       790
weighted avg       0.73      0.67      0.63       790



## Neural Network with RandomizedSearchCV

In [31]:
%%time

score_measure = "precision"
kfolds = 5

param_grid = {
    'hidden_layer_sizes': [ (50,), (70,),(50,30), (40,20), (60,40, 20), (70,50,40)],
    'activation': ['tanh', 'relu'],
    'solver': ['adam', 'sgd'],
    'alpha': [0, .2, .5, .7, 1],
    'learning_rate': ['constant', 'invscaling', 'adaptive'],
    'learning_rate_init': [0.001, 0.01, 0.1, 0.2, 0.5],
    'max_iter': [5000]
}

ann = MLPClassifier()
grid_search = RandomizedSearchCV(estimator = ann, param_distributions=param_grid, cv=kfolds, n_iter=100,
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = grid_search.fit(X_train, y_train)

bestRecallTree = grid_search.best_estimator_

print(grid_search.best_params_)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


C:\Users\Jeshwant\anaconda3\envs\tf\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
34 fits failed out of a total of 500.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
34 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Jeshwant\anaconda3\envs\tf\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Jeshwant\anaconda3\envs\tf\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py", line 749, in fit
    return self._fit(X, y, incremental=False)
  File "C:\Users\Jeshwant\anaconda3\envs\tf\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py"

{'solver': 'sgd', 'max_iter': 5000, 'learning_rate_init': 0.1, 'learning_rate': 'invscaling', 'hidden_layer_sizes': (70, 50, 40), 'alpha': 0.2, 'activation': 'relu'}
CPU times: total: 5min 8s
Wall time: 16min 49s


C:\Users\Jeshwant\anaconda3\envs\tf\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (5000) reached and the optimization hasn't converged yet.
  warnings.warn(


In [32]:
%%time
y_pred = bestRecallTree.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        51
           1       0.00      0.00      0.00        99
           2       0.00      0.00      0.00        16
           3       0.00      0.00      0.00       120
           4       0.24      1.00      0.38       186
           5       0.00      0.00      0.00       141
           6       0.00      0.00      0.00        42
           7       0.00      0.00      0.00        11
           8       0.00      0.00      0.00        99
           9       0.00      0.00      0.00        25

    accuracy                           0.24       790
   macro avg       0.02      0.10      0.04       790
weighted avg       0.06      0.24      0.09       790

CPU times: total: 93.8 ms
Wall time: 17 ms


C:\Users\Jeshwant\anaconda3\envs\tf\lib\site-packages\sklearn\utils\extmath.py:189: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
C:\Users\Jeshwant\anaconda3\envs\tf\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Jeshwant\anaconda3\envs\tf\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Jeshwant\anaconda3\envs\tf\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predict

## Neural Network Model With GridSearchCV

In [33]:
%%time

score_measure = "precision"
kfolds = 5

param_grid = {
    'hidden_layer_sizes': [ (40,20), (50,30), (50,40,30), (80,)],
    'activation': ['relu','tanh'],
    'solver': ['adam'],
    'alpha': [.3, .5, 1],
    'learning_rate': ['adaptive', 'invscaling'],
    'learning_rate_init': [0.005, 0.01, 0.15],
    'max_iter': [5000]
}

ann = MLPClassifier()
grid_search = GridSearchCV(estimator = ann, param_grid=param_grid, cv=kfolds, 
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = grid_search.fit(X_train, y_train)

bestRecallTree = grid_search.best_estimator_

print(grid_search.best_params_)

Fitting 5 folds for each of 144 candidates, totalling 720 fits


C:\Users\Jeshwant\anaconda3\envs\tf\lib\site-packages\sklearn\model_selection\_search.py:952: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(
C:\Users\Jeshwant\anaconda3\envs\tf\lib\site-packages\sklearn\model_selection\_search.py:952: UserWarning: One or more of the train scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan

{'activation': 'relu', 'alpha': 0.3, 'hidden_layer_sizes': (40, 20), 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'max_iter': 5000, 'solver': 'adam'}
CPU times: total: 3.88 s
Wall time: 1min 24s


In [34]:
%%time
y_pred = bestRecallTree.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.77      0.67      0.72        51
           1       0.87      0.78      0.82        99
           2       0.27      0.81      0.41        16
           3       0.59      1.00      0.74       120
           4       0.89      0.96      0.92       186
           5       1.00      0.03      0.06       141
           6       0.52      0.76      0.62        42
           7       0.25      0.09      0.13        11
           8       0.70      0.90      0.78        99
           9       1.00      0.20      0.33        25

    accuracy                           0.70       790
   macro avg       0.68      0.62      0.55       790
weighted avg       0.79      0.70      0.64       790

CPU times: total: 62.5 ms
Wall time: 17 ms


## Using Keras

### Deep Network

In [35]:
import os
import tensorflow as tf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [36]:
import tensorflow as tf
from tensorflow import keras

# fix random seed for reproducibility
np.random.seed(1)
tf.random.set_seed(1)

In [37]:
y_train.value_counts()

segment_encoded
4                  754
5                  559
3                  460
1                  447
8                  367
0                  252
6                  135
2                   75
9                   70
7                   41
dtype: int64

In [38]:
X_test.shape

(790, 6)

In [39]:
y_test.value_counts()

segment_encoded
4                  186
5                  141
3                  120
1                   99
8                   99
0                   51
6                   42
9                   25
2                   16
7                   11
dtype: int64

In [40]:
%%time

# create model stucture
model = keras.models.Sequential()
model.add(keras.layers.Input(6))
model.add(keras.layers.Dense(50, activation='relu'))
model.add(keras.layers.Dense(50, activation='relu'))
model.add(keras.layers.Dense(50, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax')) # final layer, 10 categories


# compile
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# if you want to overide the defaults for the optimizer....
#adam = keras.optimizers.Adam(learning_rate=0.01)
#model.compile(loss='sparse_categorical_crossentropy', optimizer=adam, metrics=['accuracy'])


CPU times: total: 141 ms
Wall time: 186 ms


In [43]:
%%time

# fit the model
history = model.fit(X_train, y_train, 
                    validation_data=(X_test, y_test), 
                    epochs=22, batch_size=100)


Epoch 1/22
32/32 [==============================] - 0s 5ms/step - loss: 0.1252 - accuracy: 0.9646 - val_loss: 0.4234 - val_accuracy: 0.9076
Epoch 2/22
32/32 [==============================] - 0s 3ms/step - loss: 1.1377 - accuracy: 0.8057 - val_loss: 0.3499 - val_accuracy: 0.8949
Epoch 3/22
32/32 [==============================] - 0s 4ms/step - loss: 0.2954 - accuracy: 0.9184 - val_loss: 0.2205 - val_accuracy: 0.8975
Epoch 4/22
32/32 [==============================] - 0s 4ms/step - loss: 0.2357 - accuracy: 0.9339 - val_loss: 0.2748 - val_accuracy: 0.9089
Epoch 5/22
32/32 [==============================] - 0s 4ms/step - loss: 0.4035 - accuracy: 0.8864 - val_loss: 0.5026 - val_accuracy: 0.8481
Epoch 6/22
32/32 [==============================] - 0s 4ms/step - loss: 0.2134 - accuracy: 0.9272 - val_loss: 0.3519 - val_accuracy: 0.8747
Epoch 7/22
32/32 [==============================] - 0s 6ms/step - loss: 0.1569 - accuracy: 0.9557 - val_loss: 0.1597 - val_accuracy: 0.9380
Epoch 8/22
32/32 [==

In [44]:
# evaluate the model

scores = model.evaluate(X_test, y_test, verbose=0)
scores
# In results, first is loss, second is accuracy

[0.47551319003105164, 0.8101266026496887]

In [45]:
# let's format this into a better output...

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

loss: 0.48
accuracy: 81.01%


## Wide and Deep Network

In [54]:
#Define the model: for multi-class

model = keras.models.Sequential()

model.add(keras.layers.Input(shape=6))
model.add(keras.layers.Dense(60, activation='relu'))
model.add(keras.layers.Dense(60, activation='relu'))
model.add(keras.layers.Dense(60, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

In [55]:
# Compile model

#Optimizer:
adam = keras.optimizers.Adam(learning_rate=0.01)
model.compile(loss='sparse_categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [60]:
# Fit the model

history = model.fit(X_train, y_train, 
                    validation_data=(X_test, y_test), 
                    epochs=40, batch_size=100)

Epoch 1/40
32/32 [==============================] - 0s 4ms/step - loss: 0.9650 - accuracy: 0.6820 - val_loss: 0.5845 - val_accuracy: 0.7658
Epoch 2/40
32/32 [==============================] - 0s 3ms/step - loss: 0.4258 - accuracy: 0.8057 - val_loss: 0.3428 - val_accuracy: 0.8722
Epoch 3/40
32/32 [==============================] - 0s 3ms/step - loss: 0.2449 - accuracy: 0.9165 - val_loss: 0.2432 - val_accuracy: 0.9291
Epoch 4/40
32/32 [==============================] - 0s 3ms/step - loss: 0.2086 - accuracy: 0.9370 - val_loss: 0.1890 - val_accuracy: 0.9481
Epoch 5/40
32/32 [==============================] - 0s 3ms/step - loss: 0.1590 - accuracy: 0.9532 - val_loss: 0.1468 - val_accuracy: 0.9570
Epoch 6/40
32/32 [==============================] - 0s 3ms/step - loss: 0.1868 - accuracy: 0.9405 - val_loss: 0.2059 - val_accuracy: 0.9152
Epoch 7/40
32/32 [==============================] - 0s 3ms/step - loss: 0.4292 - accuracy: 0.8975 - val_loss: 1.2666 - val_accuracy: 0.6747
Epoch 8/40
32/32 [==

In [61]:
# evaluate the model

scores = model.evaluate(X_test, y_test, verbose=0)
scores

# In results, first is loss, second is accuracy

[0.21578212082386017, 0.9417721629142761]

In [62]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

loss: 0.22
accuracy: 94.18%


In [63]:
## Wide and Deep network with RandomGridSearch

In [65]:
%%time

# If you don't have the following installed, from command line '!pip install scikeras'
from scikeras.wrappers import KerasClassifier
from keras.initializers import GlorotNormal

score_measure = "accuracy"
kfolds = 5

def build_clf(hidden_layer_sizes, dropout):
    ann = tf.keras.models.Sequential()
    ann.add(keras.layers.Input(shape=6)),
    for hidden_layer_size in hidden_layer_sizes:
        model.add(keras.layers.Dense(hidden_layer_size, kernel_initializer= tf.keras.initializers.GlorotNormal(), 
                                     bias_initializer=keras.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=None), activation="relu"))
        model.add(keras.layers.Dropout(dropout))
    ann.add(tf.keras.layers.Dense(10, activation='softmax'))
    ann.compile(loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
    return ann


CPU times: total: 0 ns
Wall time: 0 ns


In [66]:
from scikeras.wrappers import KerasClassifier

keras_clf = KerasClassifier(
    model=build_clf,
    hidden_layer_sizes=64,
    dropout = 0.0
)


In [67]:
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import RandomizedSearchCV

params = {
    'optimizer__learning_rate': [0.0005, 0.001, 0.005],
    'model__hidden_layer_sizes': [(70,),(90, ), (100,), (100, 90)],
    'model__dropout': [0, 0.1],
    'batch_size':[20, 60, 100],
    'epochs':[10, 50, 100],
    'optimizer':["adam",'sgd']
}
keras_clf.get_params().keys()


dict_keys(['model', 'build_fn', 'warm_start', 'random_state', 'optimizer', 'loss', 'metrics', 'batch_size', 'validation_batch_size', 'verbose', 'callbacks', 'validation_split', 'shuffle', 'run_eagerly', 'epochs', 'hidden_layer_sizes', 'dropout', 'class_weight'])

In [68]:
rnd_search_cv = RandomizedSearchCV(estimator=keras_clf, param_distributions=params, scoring='accuracy', n_iter=50, cv=5)

import sys
sys.setrecursionlimit(10000) # note: the default is 3000 (python 3.9)

earlystop = EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='auto')
callback = [earlystop]

_ = rnd_search_cv.fit(X_train, y_train, callbacks=callback, verbose=0)


11/11 [==============================] - 0s 1ms/step


In [69]:
rnd_search_cv.best_params_

{'optimizer__learning_rate': 0.005,
 'optimizer': 'sgd',
 'model__hidden_layer_sizes': (70,),
 'model__dropout': 0,
 'epochs': 100,
 'batch_size': 20}

In [70]:
best_net = rnd_search_cv.best_estimator_
print(rnd_search_cv.best_params_)

{'optimizer__learning_rate': 0.005, 'optimizer': 'sgd', 'model__hidden_layer_sizes': (70,), 'model__dropout': 0, 'epochs': 100, 'batch_size': 20}


In [71]:
%%time
y_pred = best_net.predict(X_test)
print(classification_report(y_test, y_pred))

40/40 [==============================] - 0s 1ms/step
              precision    recall  f1-score   support

           0       0.75      0.94      0.83        51
           1       0.79      0.98      0.87        99
           2       1.00      0.31      0.48        16
           3       0.97      0.96      0.96       120
           4       0.99      0.96      0.98       186
           5       0.91      0.88      0.89       141
           6       0.83      0.81      0.82        42
           7       1.00      1.00      1.00        11
           8       0.85      0.86      0.85        99
           9       1.00      0.36      0.53        25

    accuracy                           0.89       790
   macro avg       0.91      0.81      0.82       790
weighted avg       0.90      0.89      0.89       790

CPU times: total: 891 ms
Wall time: 225 ms


# Here we have considered the Neural Network, Neutral network with Random search, Neutral network with grid search, Using Keras Deep Network, Wide and Deep Network, Wide and Deep network with Randomgridsearch. 

# The Accuracy is the highest for  Decision Trees Random Search classifier	with 99.8734%